#### 가설
 - 어떤걸 올릴지 모른다.
#### API 활용
1. 현재까지 올린 영상 및 키워드 분석 후
2. 관련된 키워드를 모두 추출한다.

----- 여기부터 개별로 키워드 분석해서 해시태그 추가 조언할 수 있게 메서드화----

3. 갖고 있는 모든 리스트 키워드를 가지고 구독자 대비 조회수가 잘 뽑힌걸 추출 max 15개
4. 해당 추출된 영상 정보 가지고 추가적인 단어 군집분석?
5. 군집된 단어들 가지고 LLM 통한 주제 추천?

-> 현재까지 올린 영상 및 키워드 분석 후 최근 구독자 수 대비 조회수가 높은 점수 높은 영상들 갖고와서 화면 보여주기 및 키워드 분석, 제목 분석, 현재 트렌드 분석하여 최신 추천
#### or
어떻게 뭘 시작할지 모른다.
-> 질문 꼬리던져서 영감 주기
#### or
최신트렌드 민감
-> 인기영상 키워드 분석 + 현재 트렌드 잘 맞춰서 어떤걸 올릴지 아이디어 영감?

#### 일반 키워드 영상 조회

In [1]:
from config import Config
youtube = Config().youtube

In [25]:
def search_videos_by_keyword(youtube, keyword, maxResults):
    search_response = youtube.search().list(
        q=keyword,
        part='snippet',
        maxResults=maxResults,
        type='video'
    ).execute()
    
    video_details = []

    for item in search_response['items']:
        video_id = item['id']['videoId']
        videos = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            id=video_id
        ).execute()
        
        for video in videos['items']:
            channel_id = video['snippet']['channelId']
            channel_details = youtube.channels().list(
                part='statistics',
                id=channel_id
            ).execute()
            
            Subscribers = -1
            try:
                Subscribers = channel_details['items'][0]['statistics']['subscriberCount']
            except KeyError:
                print("Subscribers: 정보가 비공개입니다.")
                continue

            video_details.append({
                '제목': video["snippet"]["title"],
                '소유자': video["snippet"]["channelTitle"],
                '구독자수': Subscribers,
                '조회수': video["statistics"]["viewCount"],
                '영상스코어': float(video["statistics"]["viewCount"]) / float(Subscribers) if Subscribers != -1 else '알수없음',
                '썸네일': video["snippet"]["thumbnails"]["default"]["url"],
                '재생 시간': video["contentDetails"]["duration"],
                '키워드': video["snippet"]["tags"] if "tags" in video["snippet"] else [],
                '키워드수': len(video["snippet"]["tags"]) if "tags" in video["snippet"] else 0
            })

    return video_details

# 함수 호출 예시
search_videos_by_keyword(youtube, 'Python 튜토리얼', 2)

[{'제목': '파이썬 코딩 무료 강의 (기본편) - 6시간 뒤면 여러분도 개발자가 될 수 있어요 [나도코딩]',
  '소유자': '나도코딩',
  '구독자수': '340000',
  '조회수': '5153968',
  '영상스코어': 15.158729411764705,
  '썸네일': 'https://i.ytimg.com/vi/kWiCuklohdY/default.jpg',
  '재생 시간': 'PT6H1M28S',
  '키워드': ['코딩',
   '코딩교육',
   '프로그래밍',
   '파이썬',
   '자바',
   'Python',
   'Java',
   '자료구조',
   '알고리즘',
   '게임',
   '나도코딩',
   'c언어',
   '개발',
   '초보',
   '스크래치',
   '엔트리',
   'Scratch',
   'Entry',
   '3.0',
   'Pandas',
   'Numpy',
   '머신러닝',
   '딥러닝',
   '데이터',
   '데이터과학',
   '데이터분석',
   '시각화',
   '아두이노',
   'arduino',
   '사물인터넷',
   'iot',
   '기본편',
   '얼굴인식',
   'opencv',
   '업무자동화',
   'rpa',
   'gui',
   'tkinter',
   '웹크롤링',
   '웹스크래핑',
   '크롤링',
   'beautifulsoup',
   '이미지처리',
   '인공지능',
   'pyautogui',
   'matplot',
   'knn',
   '빅데이터',
   '파이썬강의',
   '파이썬강좌',
   'python 강의',
   'python 강좌',
   '파이썬 강의',
   '파이썬 강좌',
   '파이썬 교육',
   'python 교육',
   'python',
   'yt:cc=on',
   '코딩공부',
   '파이썬공부',
   '코딩 공부',
   '파이썬 공부',
   '코딩독학',
   '코딩 독학',
  

### 키워드 별 가장 인기 많은 영상 조회

In [14]:
from googleapiclient.errors import HttpError
import time

def youtube_search(youtube, q, max_results=10, order="relevance", token=None, location=None, location_radius=None, publishedAfter=None, publishedBefore=None):
    search_response = youtube.search().list(
        q=q,
        type="video",
        pageToken=token,
        order=order,
        part="id,snippet",
        maxResults=max_results,
        location=location,
        locationRadius=location_radius,
        publishedAfter=publishedAfter, #'1970-01-01T00:00:00Z'
        publishedBefore=publishedBefore #'1970-01-01T00:00:00Z'
    ).execute()

    videos = []

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":
            videos.append(search_result["id"]["videoId"])
    
    video_details = []

    for video_id in videos:
        video_response = youtube.videos().list(
            id=video_id,
            part='snippet,statistics,contentDetails'
        ).execute()

        Subscribers = -1
        for video_result in video_response.get("items", []):
            try:
                channel_details = youtube.channels().list(
                    part='statistics',
                    id=video_result['snippet']['channelId']
                ).execute()
                Subscribers = channel_details['items'][0]['statistics']['subscriberCount']
            except KeyError:
                print("Subscribers: 정보가 비공개입니다.")
            id = video_result["id"]

            # 중복제거
            if id in [video['id'] for video in video_details]:
                continue

            video_details.append({
                'id': id,
                '제목': video_result["snippet"]["title"],
                '소유자': video_result["snippet"]["channelTitle"],
                '조회수': video_result["statistics"]["viewCount"],
                '구독자수': Subscribers,
                '영상스코어': float(video_result["statistics"]["viewCount"]) / float(Subscribers) if Subscribers != -1 else '알수없음',
                '썸네일': video_result["snippet"]["thumbnails"]["default"]["url"],
                '재생 시간': video_result["contentDetails"]["duration"],
                '키워드': video_result["snippet"]["tags"] if "tags" in video_result["snippet"] else [],
                '키워드수': len(video_result["snippet"]["tags"]) if "tags" in video_result["snippet"] else 0
            })

    return video_details
# try:
#     # 키워드로 가장 인기있는 비디오 검색
#     keyword = "Python 프로그래밍"
#     videos = youtube_search(youtube, q=keyword, max_results=2, order="viewCount")
#     for video in videos:
#         print(video)
# except HttpError as e:
#     print(f"An HTTP error {e.resp.status} occurred:\n{e.content}")


#### 개별 채널 모든 영상 데이터 추출

In [2]:
from googleapiclient.errors import HttpError

def get_channel_videos(youtube, channel_name):
    # Step 1: Get the channel ID by channel name
    channel_response = youtube.search().list(
        q=channel_name,
        type="channel",
        part="id,snippet"
    ).execute()

    if not channel_response["items"]:
        print("Channel not found.")
        return

    channel_id = channel_response["items"][0]["id"]["channelId"]
    owner_name = channel_response["items"][0]["snippet"]["channelTitle"]

    # Step 2: Get channel details including subscriber count
    channel_details = youtube.channels().list(
        id=channel_id,
        part="statistics"
    ).execute()

    subscribers = channel_details["items"][0]["statistics"]["subscriberCount"]

    # Step 3: List all videos from the channel
    nextPageToken = None
    video_details = []
    while True:
        video_response = youtube.search().list(
            channelId=channel_id,
            part="id, snippet",
            maxResults=50,
            pageToken=nextPageToken
        ).execute()
        
        # Step 4: For each video, retrieve details
        for video in video_response["items"]:
            try:
                video_id = video["id"]["videoId"]
            except Exception as e:
                print('비디오가 아닙니다.')
                continue
            
            videos = youtube.videos().list(
                id=video_id,
                part="snippet,contentDetails,statistics"
            ).execute()

            for video in videos["items"]:
                video_details.append({
                    '제목': video["snippet"]["title"],
                    '소유자': video["snippet"]["channelTitle"],
                    '구독자수': subscribers,
                    '조회수': video["statistics"]["viewCount"],
                    '영상스코어': float(video["statistics"]["viewCount"]) / float(subscribers) if subscribers != -1 else '알수없음',
                    '썸네일': video["snippet"]["thumbnails"]["default"]["url"],
                    '재생 시간': video["contentDetails"]["duration"],
                    '키워드': video["snippet"]["tags"] if "tags" in video["snippet"] else [],
                    '키워드수': len(video["snippet"]["tags"]) if "tags" in video["snippet"] else 0
                })              
        nextPageToken = video_response.get('nextPageToken')
        if not nextPageToken:
            break
    
    return video_details
try:
    channel_name = "갓생종구"
    info = get_channel_videos(youtube, channel_name)
except HttpError as e:
    print(f"An HTTP error {e.resp.status} occurred: {e.content}")
info

비디오가 아닙니다.
비디오가 아닙니다.


[{'제목': '비전공자 국비코딩학원 실제 후기 - 코알못에서 3개월만에 웹개발자 취업하기 | 국비코딩학원 찾는법, 온라인 코딩 국비 학원, 장단점, 취업 빠르게 하는 팁',
  '소유자': '갓생종구',
  '구독자수': '1770',
  '조회수': '49515',
  '영상스코어': 27.97457627118644,
  '썸네일': 'https://i.ytimg.com/vi/7KiIzj0atKI/default.jpg',
  '재생 시간': 'PT5M3S',
  '키워드': ['국비코딩학원',
   '국비코딩학원후기',
   '비전공자',
   '문과생',
   '문과',
   '국비코딩',
   '국비코딩현실',
   '국비지원',
   '코딩독학'],
  '키워드수': 9},
 {'제목': '비전공자 투운사 1달 내로 단기 합격하는 후기! 인강 ㄹㅇ 노필요 - 투운사 교재 추천  | 투운사 공부법  | 비전공자 투운사  | 투운사 후기',
  '소유자': '갓생종구',
  '구독자수': '1770',
  '조회수': '4699',
  '영상스코어': 2.654802259887006,
  '썸네일': 'https://i.ytimg.com/vi/CIN6aRQh7QQ/default.jpg',
  '재생 시간': 'PT4M1S',
  '키워드': ['투자자산운용사후기',
   '투자자산운용사독학',
   '투운사독학',
   '투운사교재추천',
   '투운사공부법',
   '비전공자투운사',
   '투운사후기',
   '#갓생'],
  '키워드수': 8},
 {'제목': '호텔리어에 대한 환상  5분만에 호텔리어 현실 알려드립니다  호텔리어 그만둔 이유 | 호텔리어 실제월급',
  '소유자': '갓생종구',
  '구독자수': '1770',
  '조회수': '28998',
  '영상스코어': 16.38305084745763,
  '썸네일': 'https://i.ytimg.com/vi/qbeKFy2yuy4/default.jpg',
  '재생 시간': 'PT5M34

In [15]:
# 사용한 키워드 전부 추출
keywords = []
for video in info:
    keywords += [v.replace('#', '') for v in video['키워드']]
keywords = list(set(keywords))
keywords

# 사용한 키워드 별 영상 개수 뽑기
extracted_videos = []
for keyword in keywords[:10]:
    videos = youtube_search(youtube, q=keyword, max_results=3, order="viewCount", publishedAfter="2023-01-01T00:00:00Z")
    extracted_videos += videos

extracted_videos

[{'id': 'vRu0aW4DsKQ',
  '제목': '항공사 승무원 연봉',
  '소유자': '돈 나오는 나무',
  '조회수': '6378311',
  '구독자수': '48700',
  '영상스코어': 130.97147843942506,
  '썸네일': 'https://i.ytimg.com/vi/vRu0aW4DsKQ/default.jpg',
  '재생 시간': 'PT43S',
  '키워드': [],
  '키워드수': 0},
 {'id': '4ny-PBqwrf4',
  '제목': '솔직히, 연봉 얼마 받으세요? 직무, 연차별 현실 연봉 | 잠실',
  '소유자': '캐치TV',
  '조회수': '806675',
  '구독자수': '266000',
  '영상스코어': 3.0326127819548874,
  '썸네일': 'https://i.ytimg.com/vi/4ny-PBqwrf4/default.jpg',
  '재생 시간': 'PT7M48S',
  '키워드': ['직장인',
   '연봉',
   '현실연봉',
   '인터뷰',
   '연봉인터뷰',
   '취준생',
   '대기업',
   '중소기업',
   '취업',
   '면접',
   '자소서'],
  '키워드수': 11},
 {'id': 'C8ZT2RTEZ4Y',
  '제목': '솔직히, 연봉 얼마 받으세요? 직무, 연차별 현실 연봉 | 구로디지털단지',
  '소유자': '캐치TV',
  '조회수': '632202',
  '구독자수': '266000',
  '영상스코어': 2.3766992481203006,
  '썸네일': 'https://i.ytimg.com/vi/C8ZT2RTEZ4Y/default.jpg',
  '재생 시간': 'PT9M17S',
  '키워드': ['연봉', '퇴사', '이직', '중소기업'],
  '키워드수': 4},
 {'id': '8nkvKiT4ZlA',
  '제목': '단돈 7만원 만으로 투자자산운용사 합격할 수 있다고? 강의없이 투운사 합격한 비결 대방출',
  '소유자':

In [16]:
print(len(extracted_videos))
extracted_videos

25


[{'id': 'vRu0aW4DsKQ',
  '제목': '항공사 승무원 연봉',
  '소유자': '돈 나오는 나무',
  '조회수': '6378311',
  '구독자수': '48700',
  '영상스코어': 130.97147843942506,
  '썸네일': 'https://i.ytimg.com/vi/vRu0aW4DsKQ/default.jpg',
  '재생 시간': 'PT43S',
  '키워드': [],
  '키워드수': 0},
 {'id': '4ny-PBqwrf4',
  '제목': '솔직히, 연봉 얼마 받으세요? 직무, 연차별 현실 연봉 | 잠실',
  '소유자': '캐치TV',
  '조회수': '806675',
  '구독자수': '266000',
  '영상스코어': 3.0326127819548874,
  '썸네일': 'https://i.ytimg.com/vi/4ny-PBqwrf4/default.jpg',
  '재생 시간': 'PT7M48S',
  '키워드': ['직장인',
   '연봉',
   '현실연봉',
   '인터뷰',
   '연봉인터뷰',
   '취준생',
   '대기업',
   '중소기업',
   '취업',
   '면접',
   '자소서'],
  '키워드수': 11},
 {'id': 'C8ZT2RTEZ4Y',
  '제목': '솔직히, 연봉 얼마 받으세요? 직무, 연차별 현실 연봉 | 구로디지털단지',
  '소유자': '캐치TV',
  '조회수': '632202',
  '구독자수': '266000',
  '영상스코어': 2.3766992481203006,
  '썸네일': 'https://i.ytimg.com/vi/C8ZT2RTEZ4Y/default.jpg',
  '재생 시간': 'PT9M17S',
  '키워드': ['연봉', '퇴사', '이직', '중소기업'],
  '키워드수': 4},
 {'id': '8nkvKiT4ZlA',
  '제목': '단돈 7만원 만으로 투자자산운용사 합격할 수 있다고? 강의없이 투운사 합격한 비결 대방출',
  '소유자':

In [17]:
ranked = [video for video in extracted_videos if video['영상스코어'] >= 5]
print(len(ranked))
ranked

6


[{'id': 'vRu0aW4DsKQ',
  '제목': '항공사 승무원 연봉',
  '소유자': '돈 나오는 나무',
  '조회수': '6378311',
  '구독자수': '48700',
  '영상스코어': 130.97147843942506,
  '썸네일': 'https://i.ytimg.com/vi/vRu0aW4DsKQ/default.jpg',
  '재생 시간': 'PT43S',
  '키워드': [],
  '키워드수': 0},
 {'id': 'LY5nEuaDH5o',
  '제목': '객관식 기출문제 공부법 초현실적으로 보여드립니다. 행정사, 공인중개사, 세무사 등 국가자격시험 대비 메타인지 공부방법 회독법',
  '소유자': '널위한나의경험&행정사',
  '조회수': '833131',
  '구독자수': '31300',
  '영상스코어': 26.617603833865815,
  '썸네일': 'https://i.ytimg.com/vi/LY5nEuaDH5o/default.jpg',
  '재생 시간': 'PT30M39S',
  '키워드': ['행정사',
   '행정사공부법',
   '행정사공부',
   '행정사수험생활',
   '행정사1차공부법',
   '객관식공부법',
   '기출문제공부법',
   '행정사기출문제',
   '행정사1차기출문제',
   '기출문제객관식공부법',
   '객관식공부방법',
   '기출문제공부',
   '기출문제공부방법'],
  '키워드수': 13},
 {'id': '2KgZmkwLE2o',
  '제목': '와 미쳤다...이걸 그냥 알려죠? 실제 방법이라서 더 믿을만한 블로그 포스팅 꿀팁(인플루언서 3년차가 직접 보여줍니다)',
  '소유자': '아임플래너',
  '조회수': '220403',
  '구독자수': '16300',
  '영상스코어': 13.521656441717791,
  '썸네일': 'https://i.ytimg.com/vi/2KgZmkwLE2o/default.jpg',
  '재생 시간': 'PT18M19S',
  '키워드